In [0]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [0]:
emp = spark.createDataFrame(emp_data, emp_schema)
emp.show()

In [0]:
#extract name, age of people making above 66k
emp_2 = emp.select("name", "age", "gender", "salary").where("salary>66000")
display(emp_2)


In [0]:
emp.schema

In [0]:
from pyspark.sql.functions import expr

emp_3 = emp.select("name", emp.employee_id, expr("cast(age as int)as age"), "salary")
emp_3.printSchema()


In [0]:
#add col column_1 populate 1, column_2 populate 2
from pyspark.sql.functions import lit
emp_4 = emp.withColumn("column_1", lit("1"))\
    .withColumn("column_2", lit(2))\
        .withColumnRenamed("employee_id","emp_id")
emp_4.show()

In [0]:
emp_5 = emp_4.drop("column_2")
emp_5.show()

In [0]:
from pyspark.sql.functions import lit, col
emp_6 = emp_5.selectExpr("emp_id", \
                         "department_id",        \
                         "name",\
                         "cast(age as int) as age",\
                         "gender",\
                         "cast(salary as double) as salary", \
                         "hire_date")\
                             .withColumn("tax", col("salary")*.01)
emp_6.printSchema()

In [0]:
display(emp_6)

In [0]:
multi_cols = {
    "col1": lit('1'),
    "col2": lit(2),
    "new_tax": col("salary")*0.02
}

emp_7 = emp_6.withColumns(multi_cols)
display(emp_7)

In [0]:
emp_8 = emp_7.drop("col1", "col2")
emp_8.show()

In [0]:
from pyspark.sql.functions import when, regexp_replace

emp_9 = emp_8.withColumn("gender", \
    when(col("name")=="Jack Chan", lit(None))\
    .otherwise(col("gender")))
emp_10 = emp_9.withColumn("gen", when(col("gender")=="Male", "M")\
    .when(col("gender")=="Female", "F")\
        .otherwise("O"))\
            .drop("gender")\
                .withColumn("new_name", regexp_replace(col("name"),'J','Z'))
emp_10.show()


In [0]:
#date play
from pyspark.sql.functions import to_date, datetimestamp

emp_11 = emp_10.withColumn("new_date", to_date("hire_date",  'yyyy-MM-dd'))
emp_11.show()